# Introdução à Banco de Dados | Trabalho Prático III
Abel Severo Rocha, Ana Carla Fernandes, Natasha Caxias
Prof. Dr. Altigran Soares da Silva

## Parte I
Verificação dos parâmetros do hardware e do software utilizado e familiarização com o ambiente do PostgreSQL.

### Tarefa 1 - Identificação do Sistema

In [1]:
import platform
import os
import shutil
import subprocess
import datetime
import psycopg2
import pandas as pd
from tabulate import tabulate
from psycopg2.extras import RealDictCursor
from tpch_pgsql import main as tpch_pgsql
from tpch4pgsql import postgresqldb as pgdb, result as r

In [2]:
def mostrarSoInfo():
    
    soInfo = platform.freedesktop_os_release()
    print("\n--------- Sistema Operacional (SO) ---------\n")
    print("Sistema Operacional ", platform.system())
    print("Versão              ", soInfo["VERSION"])
    print("Release (kernel)    ", platform.release())
    print("Arquitetura         ", platform.machine())
    print("Distribuição        ", soInfo["NAME"])


In [3]:
def mostrarHardwareInfo():

    cpuInfo = {}
    with open("/proc/cpuinfo") as f:
        for line in f:
            if ":" in line:
                k, v = line.split(":", 1)
                cpuInfo[k.strip()] = v.strip()
    
    ramInfo = {}
    with open("/proc/meminfo") as f:
        for line in f:
            k, v = line.split(":", 1)
            ramInfo[k] = v.strip()

    print("\n------------- Sobre o Hardware -------------\n")
    print("Processador:")
    print("  • Modelo          ", cpuInfo["model name"])
    print("  • CPUs lógicas    ", os.cpu_count())
    print("  • Clock base      ", cpuInfo["cpu MHz"], "MHz")

    print("\nCache:")
    with open("/sys/devices/system/cpu/cpu0/cache/index0/size") as f:
        print("  • L1d             ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index1/size") as f:
        print("  • L1i             ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index2/size") as f:
        print("  • L2              ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index3/size") as f:
        print("  • L3              ", f.read().strip()[:-1]+" kB")

    print("\nMemória RAM:")
    print("  • Total           ", ramInfo["MemTotal"])
    print("  • Livre           ", ramInfo["MemFree"])
    print("  • Disponível      ", ramInfo["MemAvailable"])

    total, usado, livre = shutil.disk_usage("/")
    print("\nDisco:")
    print(f"  • Total            {total/1024**3:.2f} GB")
    print(f"  • Usado            {usado/1024**3:.2f} GB")
    print(f"  • Livre            {livre/1024**3:.2f} GB")


In [4]:
mostrarSoInfo()
mostrarHardwareInfo()


--------- Sistema Operacional (SO) ---------

Sistema Operacional  Linux
Versão               24.04.3 LTS (Noble Numbat)
Release (kernel)     6.8.0-87-generic
Arquitetura          x86_64
Distribuição         Ubuntu

------------- Sobre o Hardware -------------

Processador:
  • Modelo           11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz
  • CPUs lógicas     8
  • Clock base       3933.273 MHz

Cache:
  • L1d              48 kB
  • L1i              32 kB
  • L2               1280 kB
  • L3               12288 kB

Memória RAM:
  • Total            7670428 kB
  • Livre            285960 kB
  • Disponível       1430880 kB

Disco:
  • Total            233.18 GB
  • Usado            54.38 GB
  • Livre            166.88 GB


### Tarefa 2 - Verificação de parâmetros de armazenamento

In [ ]:
# Permite que os acessos de hdparm sejam feitos sem sudo
# sudo visudo
# seu_usuario ALL=(ALL) NOPASSWD: /sbin/hdparm

In [12]:
BASE = "/sys/block"

def run(cmd):
    return subprocess.check_output(cmd, capture_output=True, text=True).strip()

def getDispositivo():
    for dev in os.listdir(BASE):
        if dev.startswith(("loop", "ram", "dm-")): # ignora
            continue
        return dev # encontra dispositivo
    return None

def ehHD(dispositivo):
    rotational_path = os.path.join(BASE, dispositivo, "queue/rotational")
    
    if os.path.exists(rotational_path):
        with open(rotational_path) as f:
            return f.read().strip() == "1"
    return False

def blocos(dispositivo):
    cmd = f"cat /sys/block/{dispositivo}/queue/logical_block_size"
    print("Blocos Lógicos:", run(cmd.split()))

    cmd = f"cat /sys/block/sdX/queue/physical_b"
    print("Blcocos Físicos: ", run(cmd.split()))

def info(dispositivo):
    cmd = f"sudo hdparm -I /dev/{dispositivo}"
    print(run(cmd.split()))

def stat(dispositivo):
    cmd = f"stat -f /dev/{dispositivo}"
    print("Parâmetros do SO para o disco:\n")
    print(run(cmd.split()))

def mostrarDiscoInfo():
    disp = getDispositivo()
    print("Tipos de dispositivo:", "HD" if ehHD(disp) else "SSD")
    print("Modelo", run(["cat", f"/sys/block/{disp}/device/model"]))
    #info(disp)
    stat(disp)
    blocos(disp)

In [14]:
disp = getDispositivo()
print(disp)
if ehHD(disp):
    mostrarDiscoInfo()
else:
    print("Dispositivo de armazenamento é um SSD.")

nvme0n1
Dispositivo de armazenamento é um SSD.


### Tarefa 3 – Geração de um BD para testes


Necessário dar permissão de escrita para a pasta _tpch4pgsql_:

``chmod -R u+rwX,g+rwX,o+rX .``

#### Preparar
A fase de preparação compila o TPC-H dbgen e querygen e cria os arquivos de carga e atualização (atualização/exclusão).

In [8]:
tpch_pgsql(phase="prepare")

make: Nothing to be done for 'all'.
built dbgen from source


TPC-H Population Generator (Version 2.14.0)
Copyright Transaction Processing Performance Council 1994 - 2010
Generating data for suppliers table
Preloading text ...                                                                                                                                                                                                                                                                                                              1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 

generated data for the load phase


TPC-H Population Generator (Version 2.14.0)
Copyright Transaction Processing Performance Council 1994 - 2010
Generating update pair #1 for orders/lineitem tables
Preloading text ...                                                                                                                                                                                                                                                                                                              1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2

generated data for the update phase
generated data for the delete phase
created data files in ./data
created query files in ./query_root


done.
Generating update pair #2 for orders/lineitem tablesdone.
Generating update pair #3 for orders/lineitem tablesdone.


#### Carregamento

A fase de carregamento (load) limpa o banco de dados (se necessário), carrega as tabelas no banco de dados e cria índices para consultas. Os resultados desta fase consistem nas seguintes métricas:

* Tempo de criação do esquema
* Tempo de carregamento dos dados
* Tempo de criação de restrições de chave estrangeira e índices

In [9]:
tpch_pgsql(phase="load")

dropped existing tables
cleaned database tpch
done creating schemas
done loading data to tables
done creating indexes and foreign keys
=========================== Load ===========================
create_schema: : 0:00:00.074149
load_data: 0:00:16.745466
index_tables: 0:00:18.893954
======================= End Results ========================


#### Consultas

Nessa fase, as consultas serão analizadas com o comando ``EXPLAIN ANALYZE``, evidenciando:

* Tempo de execução do planejamento;
* Tempo de execução do _exlpain_;
* Algoritmos utilizados.

Em seguida, as consultas serão executas, exibindo as seguintes informações:

* Até 10 linhas do resultado da consulta;
* Total de linhas encontradas;
* Tempo de execução.

In [10]:
class PGDBWithResults(pgdb.PGDB):

    def __init__(self, host, port, database, user, password):
        # Chamar o construtor da classe pai corretamente
        self.conn = psycopg2.connect(
                    host=host,
                    port=port,
                    dbname=database,
                    user=user,
                    password=password
                ) 
        if hasattr(self, 'conn') and self.conn:
            print("Conexão estabelecida com sucesso!")
        else:
            print("Atenção: Conexão não estabelecida")
    
    def executeQueryFromFileWithResults(self, filepath):
        """Executa query de arquivo e retorna resultados"""
        try:
            # Verificar se a conexão existe
            if not hasattr(self, 'conn') or not self.conn:
                return {'error': 'Conexão não disponível'}
            
            with open(filepath, 'r') as f:
                query = f.read()
            
            # Usar cursor que retorna dicionários
            with self.conn.cursor(cursor_factory=RealDictCursor) as cursor:
                cursor.execute(query)
                
                if cursor.description:  # Se é uma query SELECT
                    results = cursor.fetchall()
                    columns = [desc[0] for desc in cursor.description]
                    return {
                        'columns': columns,
                        'data': results,
                        'rowcount': cursor.rowcount
                    }
                else:  # Para INSERT, UPDATE, DELETE
                    return {
                        'rowcount': cursor.rowcount,
                        'message': f"Query executada: {cursor.rowcount} linhas afetadas"
                    }
                    
        except Exception as e:
            return {'error': str(e)}
        
    def executeQuery(self, query_string):
        """Executa uma query diretamente a partir de string"""
        try:
            if not self.conn:
                return {'error': 'Conexão não disponível'}
            
            with self.conn.cursor(cursor_factory=RealDictCursor) as cursor:
                cursor.execute(query_string)
                
                if cursor.description:  # Se é uma query SELECT
                    results = cursor.fetchall()
                    columns = [desc[0] for desc in cursor.description]
                    return {
                        'columns': columns,
                        'data': results,
                        'rowcount': cursor.rowcount
                    }
                else:  # Para INSERT, UPDATE, DELETE
                    self.conn.commit()
                    return {
                        'rowcount': cursor.rowcount,
                        'message': f"Query executada: {cursor.rowcount} linhas afetadas"
                    }
                    
        except Exception as e:
            return {'error': str(e)}

In [11]:
host = "localhost"
port = 5432
user = "postgres"
password = "test123"
database = "tpch"
filepath = "query_root/perf_query_gen/"

conn = PGDBWithResults(host, port, database, user, password)

# Definir algoritmos de junção e varredura comuns no PostgreSQL
algoritmos_juncao = ['Nested Loop', 'Hash Join', 'Merge Join']
algoritmos_varredura = ['Seq Scan', 'Index Scan', 'Index Only Scan', 'Bitmap Heap Scan', 'Bitmap Index Scan']
algoritmos_ordenacao = ['Sort', 'Incremental Sort']
algoritmos_agregacao = ['HashAggregate', 'GroupAggregate']

for i in [1, 3, 5, 6, 7, 9, 10, 12]:
    print(f"\n{'=' * 120}")
    print(f"EXPLAIN ANALYZE - QUERY {i}")
    print(f"{'=' * 120}")
    
    # Ler a query original
    with open(filepath + f"{i}.sql", 'r') as f:
        original_query = f.read()
    
    # Adicionar EXPLAIN ANALYZE
    explain_query = "EXPLAIN ANALYZE " + original_query
    
    start_time = datetime.datetime.now()
    result = conn.executeQuery(explain_query)
    end_time = datetime.datetime.now()
    execution_time = end_time - start_time
    
    if 'error' in result:
        print(f"Erro no EXPLAIN ANALYZE {i}: {result['error']}")
    elif 'data' in result:
        print(f"\nTempo de execução do EXPLAIN: {execution_time}")
        
        # Coletar algoritmos utilizados
        algoritmos_encontrados = {
            'juncao': [],
            'varredura': [],
            'ordenacao': [],
            'agregacao': [],
            'outros': []
        }
        
        print(f"\nPLANO DE EXECUÇÃO (EXPLAIN ANALYZE):")
        for idx, row in enumerate(result['data']):
            plan_line = list(row.values())[0] if row else ""
            print(f"{plan_line}")
            
            # Identificar algoritmos na linha do plano
            for algoritmo in algoritmos_juncao:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['juncao']:
                        algoritmos_encontrados['juncao'].append(algoritmo)
            
            for algoritmo in algoritmos_varredura:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['varredura']:
                        algoritmos_encontrados['varredura'].append(algoritmo)
            
            for algoritmo in algoritmos_ordenacao:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['ordenacao']:
                        algoritmos_encontrados['ordenacao'].append(algoritmo)
            
            for algoritmo in algoritmos_agregacao:
                if algoritmo in plan_line:
                    if algoritmo not in algoritmos_encontrados['agregacao']:
                        algoritmos_encontrados['agregacao'].append(algoritmo)
        
        # Mostrar resumo dos algoritmos utilizados
        print(f"\n{'─' * 80}")
        print("ALGORITMOS IDENTIFICADOS NO PLANO DE EXECUÇÃO:")
        print(f"{'─' * 80}")
        
        if algoritmos_encontrados['juncao']:
            print(f"Algoritmos de Junção: {', '.join(algoritmos_encontrados['juncao'])}")
        
        if algoritmos_encontrados['varredura']:
            print(f"Algoritmos de Varredura: {', '.join(algoritmos_encontrados['varredura'])}")
        
        if algoritmos_encontrados['ordenacao']:
            print(f"Algoritmos de Ordenação: {', '.join(algoritmos_encontrados['ordenacao'])}")
        
        if algoritmos_encontrados['agregacao']:
            print(f"Algoritmos de Agregação: {', '.join(algoritmos_encontrados['agregacao'])}")
        
        # Verificar se não foram encontrados algoritmos conhecidos
        total_algoritmos = (len(algoritmos_encontrados['juncao']) + 
                          len(algoritmos_encontrados['varredura']) + 
                          len(algoritmos_encontrados['ordenacao']) + 
                          len(algoritmos_encontrados['agregacao']))
        
        if total_algoritmos == 0:
            print("ℹNenhum algoritmo específico identificado (possivelmente plano simples)")
            
    else:
        print(f"{result.get('message', 'Resultado inesperado')}")
        print(f"Tempo: {execution_time}")

Conexão estabelecida com sucesso!

EXPLAIN ANALYZE - QUERY 1

Tempo de execução do EXPLAIN: 0:00:03.003160

PLANO DE EXECUÇÃO (EXPLAIN ANALYZE):
Limit  (cost=346484.38..346485.51 rows=1 width=248) (actual time=2802.395..2803.793 rows=1 loops=1)
  ->  Finalize GroupAggregate  (cost=346484.38..391574.69 rows=40000 width=248) (actual time=2792.252..2793.650 rows=1 loops=1)
        Group Key: l_returnflag, l_linestatus
        ->  Gather Merge  (cost=346484.38..387874.69 rows=80000 width=248) (actual time=2780.618..2793.574 rows=4 loops=1)
              Workers Planned: 2
              Workers Launched: 2
              ->  Partial GroupAggregate  (cost=345484.36..377640.68 rows=40000 width=248) (actual time=1354.088..2294.877 rows=3 loops=3)
                    Group Key: l_returnflag, l_linestatus
                    ->  Sort  (cost=345484.36..347568.11 rows=833502 width=144) (actual time=925.820..1151.124 rows=1487020 loops=3)
                          Sort Key: l_returnflag, l_linestatu

## Parte II
Análise do comportamento dos índices das tabelas do SGBD através do exame das tabelas de estatísticas para consultas SQL.